## MNIST 

MNIST jest dużym zbiorem ręcznie pisanych cyfr.

https://en.wikipedia.org/wiki/MNIST_database

## MNIST

MNIST is a large collection of handwritten numbers.

https://en.wikipedia.org/wiki/MNIST_database

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets

In [ ]:
def plot10(x,**args):
    f,ax = plt.subplots(2,5,figsize=(10,4))
    for i,c_ in enumerate(x):
        ax.ravel()[i].imshow( np.abs(c_).reshape(28,28),**args ) 
    plt.show()

### Wczytanie danych MNIST

Jest wiele sposobów załadowania danych mnist do pamięci. Możemy skorzystać z funkcji `fetch_openml`.




In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [ ]:
X.dtype,y.shape

In [ ]:
np.prod(X.shape)/1024**2

### Zadanie 1.

Dokonaj podziału danych i etykiet na dwa zbiory - trenujący i testowy o rozmiarach odpowiednio `60000` i `10000`.

### Excercise 1.

Split the data and labels into two sets - training and test with sizes `60000` and `10000` respectively.

In [ ]:
### BEGIN SOLUTION

(train, test, train_labels, test_labels) = \
  train_test_split(X.astype(np.float32), y, \
                   test_size=1/7., random_state=42)

### END SOLUTION


In [ ]:
assert train.shape == (60000, 784)
assert test.shape == (10000, 784)
assert train_labels.shape == (60000,)
assert test_labels.shape == (10000,)
assert train[123].dtype == np.float32
assert train[123].shape == (784,)

### Zadanie 2.

Wyświetl dla losowego przykładu etykietę oraz cyfrę. Zauważ, że cyfry są obrazkami `28x28`.

### Exercise 2.

Display the label and number for a random example. Note that the numbers are `28x28` images.

In [ ]:
### BEGIN SOLUTION
ith = 1234
plt.imshow(train[ith].reshape(28,28))
print(train_labels[ith])
### END SOLUTION


## MNIST - linear model

## MNIST - linear model

In [ ]:

### BEGIN SOLUTION

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(multi_class='ovr',solver='liblinear',C=.1)
%time clf.fit(train[:800],train_labels[:800])
%time print(clf.score(test,test_labels ))

### END SOLUTION


In [ ]:
train.nbytes/1024**2

In [ ]:
# reduce examples
train,train_labels = train[:1000],train_labels[:1000]

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log',max_iter=45000, tol=1e-4,n_jobs=4)
%time clf.fit(train, train_labels)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(54,)
pca.fit(train)

red_train = pca.transform(train)
red_test = pca.transform(test)

plot10(pca.inverse_transform(red_train)[:10])
plot10(train[:10])

In [ ]:
clf = LogisticRegression(multi_class='ovr',solver='lbfgs',C=.1,n_jobs=1)
red_train = pca.transform(train)

%time clf.fit(red_train,train_labels)
%time print(clf.score(red_test,test_labels ))


## Perceptron 

In [ ]:
from sklearn.linear_model import Perceptron

clf = Perceptron(tol=1e-5, random_state=0)
%time clf.fit(train, train_labels)

In [ ]:
%time print(clf.score(test,test_labels ))


## MNIST z klasyfikatorem kNN

Korzystając z pakietu `sklearn` poklasyfikuj dane MNIST.

*UWAGA* -  testowanie klasyfikatora wymaga kilku sekund na 100 przykładów. Dlatego funkcje: `clf.score` i `clf.predict` należy wywoływać na podziorze danych testujących lub liczyć się z długim czasem działania. 

## MNIST with the kNN classification

Use the `sklearn` package to classify MNIST data.

*NOTE* - classifier testing requires a few seconds for 100 examples. Therefore, the functions `clf.score` and` clf.predict` should be called on the test data subdivision or a long operating time should be taken into account.

In [ ]:
### BEGIN SOLUTION

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(7,n_jobs=2)
clf.fit(train,train_labels)
%time print(clf.score(test[:1000],test_labels[:1000] ))

### END SOLUTION


### Predykcja

Znajdź przykłady (np. 10 sztuk) dla których klasyfikator się pomylił. Zastanów się czy człowiek też mógłby się pomylić w tych przypadkach.

### Prediction

Find examples (e.g. 10 items) for which the classifier was wrong. Consider whether man could also be mistaken in these cases.

In [ ]:
test_labels[123]

In [ ]:
### BEGIN SOLUTION
for ith in range(100):
    predicted = int(clf.predict( test[ith:ith+1])[0])
    actual = int(test_labels[ith]) 
    if predicted != actual:
        print(predicted,actual)
        plt.imshow(test[ith].reshape(28,28))
        plt.show()
### END SOLUTION


##  MNIST z klasyfikatorem SVM

Poklasyfikuj dane MNIST klasyfikatorem SVM, korzystając z implementacji `sklearn` 

 - należy zastosować skalowanie danych (np. `preprocessing.StandardScaler`)
 - warto zacząć eksperymenty od `1000` przykładów trenujących. Trenowanie SVM jest wolne.

## MNIST with SVM classifier

Classify the MNIST data with an SVM classifier using the implementation of `sklearn`

 - data scaling should be used (e.g. `preprocessing.StandardScaler`)
 - it is worth starting experiments with '1000' training examples. SVM training is slow.

In [ ]:
### BEGIN SOLUTION

from sklearn import svm
from sklearn import preprocessing
scaler = preprocessing.StandardScaler(with_std = True)
scaler.fit(train)
clf = svm.SVC(decision_function_shape='ovo',kernel='rbf')

%time clf.fit(scaler.transform(train[:10000]),train_labels[:10000])

clf.score(scaler.transform(test[:1333]),test_labels[:1333])

### END SOLUTION


##  MNIST z klasyfikatorem SVM w zredukowanej przestrzeni danych (PCA)

Wykonaj redukcję danych z pomocą PCA, np.:

    pca = decomposition.PCA(n_components=50, whiten=True,\
                            svd_solver='randomized')
                            
Zmniejszając wymiar przestrzeni można pokusić się o zwiększenie zbioru trenującego!

## MNIST with SVM classifier in reduced data space (PCA)

Perform data reduction with the help of PCA, e.g .:

    pca = decomposition.PCA (n _components = 50, whiten = True, \
                            svd_ = solver 'randomized')
                            
By reducing the space dimension, you can try to increase the training set!

In [ ]:
### BEGIN SOLUTION

from sklearn import  decomposition

pca = decomposition.PCA(n_components=35, whiten=True,\
                        svd_solver='randomized')
%time pca.fit(train)
train_pca = pca.transform(train)
test_pca = pca.transform(test)

%time clf.fit(train_pca,train_labels)

%time print(clf.score(test_pca[:],test_labels[:]))

### END SOLUTION


### testing linear solvers
    solvers = {}
    for s in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        clf = LogisticRegression(multi_class='ovr',solver=s,C=.1)

        hist= [] 
        for i in range(1,100):

            if 2**i > 4100:
                break
            t = time.time()
            clf.fit(train[:2**i],train_labels[:2**i])
            hist.append( (2**i, time.time() - t) )
            print(2**i,s)
        hist = np.array(hist)
        solvers[clf.solver] = hist.copy()

    for n,h in solvers.items():
        plt.loglog(h[:,0],h[:,1],'o-',label=n)
    plt.legend()
    plt.show()